## TRK to USD

### As streamlines

In [ ]:
import nibabel as nib
import numpy as np
from pxr import Usd, UsdGeom, Gf
import os
import glob

def load_trk_files_from_folder(folder_path):
    # Find all .trk files in the specified folder
    trk_files = glob.glob(os.path.join(folder_path, "*.trk"))
    return trk_files

def calculate_orientation_color(point1, point2):
    # Calculate the vector between two points and map it to RGB color
    vector = point2 - point1
    norm = np.linalg.norm(vector)
    if norm == 0:
        return Gf.Vec3f(1.0, 1.0, 1.0)  # Default color for zero-length vectors
    direction = vector / norm
    color = Gf.Vec3f(float(0.5 + 0.5 * direction[0]), float(0.5 + 0.5 * direction[1]), float(0.5 + 0.5 * direction[2]))
    return color

def create_usd_with_streamlines_from_files(trk_files, usd_file_path):
    # Create USD file to hold streamlines from multiple files
    stage = Usd.Stage.CreateNew(usd_file_path)
    root = UsdGeom.Xform.Define(stage, "/Streamlines")

    for file_index, trk_file in enumerate(trk_files):
        # Load streamlines for the current file
        tractogram = nib.streamlines.load(trk_file)
        streamlines = tractogram.streamlines
        
        # Create an Xform group for this file
        file_name = os.path.basename(trk_file).replace('.trk', '')
        file_group = UsdGeom.Xform.Define(stage, f"/Streamlines/{file_name}")
        
        # Define a BasisCurves object for the streamlines in this file
        usd_curves = UsdGeom.BasisCurves.Define(stage, f"/Streamlines/{file_name}/Streamlines")
        usd_curves.GetTypeAttr().Set(UsdGeom.Tokens.linear)  # Set curve type to linear for polyline

        all_points = []
        curve_vertex_counts = []
        all_colors = []
        widths = []

        # Iterate through each streamline to add vertices, colors, and widths
        for streamline in streamlines:
            curve_vertex_counts.append(len(streamline))  # Number of vertices in this curve
            for i in range(len(streamline) - 1):
                point1 = streamline[i]
                point2 = streamline[i + 1]
                
                # Convert point1 to Gf.Vec3f and add to all_points
                all_points.append(Gf.Vec3f(float(point1[0]), float(point1[1]), float(point1[2])))
                
                # Calculate color based on orientation between point1 and point2
                color = calculate_orientation_color(point1, point2)
                all_colors.append(color)
                
                # Set the width for this vertex
                widths.append(0.1)  # Set radius (thickness) to 0.1

            # Add the last point of the streamline
            last_point = streamline[-1]
            all_points.append(Gf.Vec3f(float(last_point[0]), float(last_point[1]), float(last_point[2])))
            all_colors.append(color)  # Use the last color calculated for the final point
            widths.append(0.1)  # Maintain width for the last point

        # Set points, curve vertex counts, colors, and widths for the BasisCurves object
        usd_curves.GetPointsAttr().Set(all_points)
        usd_curves.GetCurveVertexCountsAttr().Set(curve_vertex_counts)
        
        # Set per-vertex color using Primvar
        color_attr = usd_curves.CreateDisplayColorPrimvar(UsdGeom.Tokens.varying)
        color_attr.Set(all_colors)
        
        # Set widths to simulate thickness
        usd_curves.GetWidthsAttr().Set(widths)

    # Save the stage
    stage.GetRootLayer().Save()

def combine_trk_folder_to_usd(folder_path, usd_file_path):
    # Get all .trk files in the specified folder
    trk_files = load_trk_files_from_folder(folder_path)
    
    # Export each file's streamlines as separate entries in the USD file
    create_usd_with_streamlines_from_files(trk_files, usd_file_path)

In [ ]:
# Example usage
folder_path = "/media/favreau/Documents/medias/dti/HCP_subject124422_100Kseeds"
usd_file = "/home/favreau/medias/usd/dti/HCP_subject124422_100Kseeds_as_streamlines.usd"
combine_trk_folder_to_usd(folder_path, usd_file)

In [3]:
# Example usage
folder_path = "/media/favreau/Documents/medias/dti/HCP_subject124422_3Mseeds"
usd_file = "/home/favreau/medias/usd/dti/HCP_subject124422_3Mseeds_as_streamlines.usd"
combine_trk_folder_to_usd(folder_path, usd_file)

### As points

In [4]:
import nibabel as nib
import numpy as np
from pxr import Usd, UsdGeom, Gf
import os
import glob

def load_trk_files_from_folder(folder_path):
    # Find all .trk files in the specified folder
    trk_files = glob.glob(os.path.join(folder_path, "*.trk"))
    return trk_files

def calculate_orientation_color(point1, point2):
    # Calculate the vector between two points and map it to RGB color
    vector = point2 - point1
    norm = np.linalg.norm(vector)
    if norm == 0:
        return Gf.Vec3f(1.0, 1.0, 1.0)  # Default color for zero-length vectors
    direction = vector / norm
    color = Gf.Vec3f(float(0.5 + 0.5 * direction[0]), float(0.5 + 0.5 * direction[1]), float(0.5 + 0.5 * direction[2]))
    return color

def create_usd_with_points_from_files(trk_files, usd_file_path):
    # Create USD file to hold points from multiple files
    stage = Usd.Stage.CreateNew(usd_file_path)
    root = UsdGeom.Xform.Define(stage, "/Streamlines")

    for file_index, trk_file in enumerate(trk_files):
        # Load streamlines for the current file
        tractogram = nib.streamlines.load(trk_file)
        streamlines = tractogram.streamlines
        
        # Create an Xform group for this file
        file_name = os.path.basename(trk_file).replace('.trk', '')
        file_group = UsdGeom.Xform.Define(stage, f"/Streamlines/{file_name}")
        
        # Define a Points object for the streamlines in this file
        usd_points = UsdGeom.Points.Define(stage, f"/Streamlines/{file_name}/Points")
        
        all_points = []
        all_colors = []
        widths = []

        # Iterate through each streamline to add points
        for streamline in streamlines:
            for i in range(len(streamline) - 1):
                point1 = streamline[i]
                point2 = streamline[i + 1]
                
                # Add point1 to all_points
                all_points.append(Gf.Vec3f(float(point1[0]), float(point1[1]), float(point1[2])))
                
                # Calculate color based on orientation between point1 and point2
                color = calculate_orientation_color(point1, point2)
                all_colors.append(color)
                
                # Set a uniform width for points
                widths.append(0.1)  # Adjust as needed for visibility

        # Set points, colors, and widths for the USD Points object
        usd_points.GetPointsAttr().Set(all_points)
        usd_points.GetWidthsAttr().Set(widths)
        
        # Set per-point color using Primvar
        color_attr = usd_points.CreateDisplayColorPrimvar(UsdGeom.Tokens.varying)
        color_attr.Set(all_colors)
        
        # Ensure visibility
        usd_points.GetVisibilityAttr().Set(UsdGeom.Tokens.inherited)

    # Save the stage
    stage.GetRootLayer().Save()

def combine_trk_folder_to_usd(folder_path, usd_file_path):
    # Get all .trk files in the specified folder
    trk_files = load_trk_files_from_folder(folder_path)
    
    # Export each file's streamlines as separate entries in the USD file
    create_usd_with_points_from_files(trk_files, usd_file_path)

In [3]:
# Example usage
folder_path = "/media/favreau/Documents/medias/dti/ISMRM_2015_Tracto_challenge_ground_truth_bundles_TRK_v2"
usd_file = "/home/favreau/medias/usd/dti/ISMRM_2015_Tracto_challenge_ground_truth_bundles_TRK_v2.usd"
combine_trk_folder_to_usd(folder_path, usd_file)

In [5]:
# Example usage
folder_path = "/media/favreau/Documents/medias/dti/HCP_subject124422_100Kseeds"
usd_file = "/home/favreau/medias/usd/dti/HCP_subject124422_100Kseeds.usd"
combine_trk_folder_to_usd(folder_path, usd_file)

In [4]:
# Example usage
folder_path = "/media/favreau/Documents/medias/dti/HCP_subject124422_3Mseeds"
usd_file = "/home/favreau/medias/usd/dti/HCP_subject124422_3Mseeds.usd"
combine_trk_folder_to_usd(folder_path, usd_file)